In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [3]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import random
import pickle

emotion_scores = {
    "sad" : 0,
    "joy" : 0, 
    "angry" : 0, 
    "fear" : 0,
    "greetings" : 0, 
    "farewell" : 0,
    "thanks" : 0
}

with open("dialogflowintents.json") as file:
    data = json.load(file)

def respond(tag):
  for i in data['intents']:
    if i['tag'] == tag:
      increment_score(tag)
      print("ChatBot:", np.random.choice(i['responses']))

def increment_score(tag):
    emotion_scores[tag[0]]+=1
    print(emotion_scores[tag[0]])

def get_highest_key():
  highest_value = 0
  for key in emotion_scores.keys():
    if emotion_scores[key] >= highest_value:
      highest_value = emotion_scores[key]
      highest_key = key
  return highest_key

def consolidation_message(highest_key):
    if highest_key == 'sad':
      print("ChatBot: You seem to be really sad, maybe consult a therapist or talk to a close friend")
    elif highest_key == 'joy':
      print("You seem to be in a good mood, hope you feel this way always!")
    elif highest_key == 'angry':
      print("You seem agitated right now, take a deep breath")
    elif highest_key == 'fear':
      print("I understand you feel scared, this too shall pass.")
    else:
      print("Goodbye! Take care")

def chat():
    # load trained model
    model = keras.models.load_model('therabot_dfintents_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)
    
    while True:
        print("User: ", end="")
        inp = input()
        if inp.lower() == "quit":
            highest_key = get_highest_key()
            consolidation_message(highest_key)
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=20))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        respond(tag)

print("Start messaging with the bot (type quit to stop)!")
chat()

Start messaging with the bot (type quit to stop)!
User: hi
1
ChatBot: Hi
User: hey
2
ChatBot: Hi there
User: im sad
1
ChatBot: You are sad
User: im not good
1
ChatBot: You are worried
User: im unhappy
2
ChatBot: You are sad
User: im depressed
3
ChatBot: You are sad
User: quit
ChatBot: You seem to be really sad, maybe consult a therapist or talk to a close friend
